## Pipeline de Processamento e Armazenamento de Embeddings

Este pipeline foi projetado para **extrair, limpar e converter informações acadêmicas** de arquivos JSON em **representações vetoriais (embeddings)**, que são então armazenadas para futuras consultas semânticas. O foco é preparar uma base de dados vetorial robusta para sistemas de busca e aplicações de Recuperação Aumentada por Geração (RAG).

### Ferramentas Principais

* **Manipulação de arquivos e dados**: Bibliotecas Python padrão (`os`, `json`, `pathlib`, etc.).
* **Geração de Embeddings**: [sentence-transformers](https://www.sbert.net/) para a criação de vetores semânticos.
* **Armazenamento Vetorial**: [ChromaDB](https://www.trychroma.com/) para gerenciar e indexar os embeddings.

### Estratégia de Chunking e Metadata

Cada arquivo JSON é segmentado em **chunks semânticos**, garantindo que as informações relevantes sejam agrupadas de forma coesa. Alguns exemplos de agrupamentos incluem:

* Dados básicos do professor (nome, departamento, período).
* Disciplinas ministradas (nome, código, turma, formato).
* Horários detalhados por disciplina e dia da semana.

Cada chunk é enriquecido com **metadados** que facilitam buscas e filtragens futuras, como o nome do professor ou o código da disciplina.

### Modelos de Embeddings Utilizados

Para cada chunk, embeddings são gerados usando uma variedade de modelos da família SentenceTransformer, permitindo flexibilidade e comparação de desempenho:

* `all-MiniLM-L6-v2`
* `all-mpnet-base-v2`
* `paraphrase-multilingual-MiniLM-L12-v2`
* `distiluse-base-multilingual-cased-v2`
* `stsb-xlm-r-multilingual`
* `bert-large-portuguese-cased` (neuralmind)

### Armazenamento no ChromaDB

Os **embeddings, textos originais e seus metadados** são organizados e armazenados em coleções separadas no ChromaDB. Cada modelo de embedding possui sua própria coleção, o que permite avaliar e comparar a performance de diferentes modelos em consultas subsequentes.

### Fluxo do Pipeline

1.  **Leitura**: Ingestão de arquivos JSON de professores.
2.  **Chunking**: Divisão dos dados em unidades semânticas com metadados.
3.  **Embeddings**: Geração de vetores para cada chunk usando todos os modelos definidos.
4.  **Armazenamento**: Persistência dos embeddings, textos e metadados nas respectivas coleções do ChromaDB.

In [ ]:
# Célula 1: Importando bibliotecas
import os
import json
import numpy as np
from typing import List, Dict, Tuple
from pathlib import Path
import chromadb
from sentence_transformers import SentenceTransformer
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.decomposition import PCA

In [2]:
# Célula 2: Configuração dos Modelos de Embeddings
def setup_embedding_models() -> List[Tuple[str, SentenceTransformer]]:
    models = [
        ("all-MiniLM-L6-v2", SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')),
        ("all-mpnet-base-v2", SentenceTransformer('sentence-transformers/all-mpnet-base-v2')),
        ("paraphrase-multilingual-MiniLM-L12-v2", SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')),
        ("distiluse-base-multilingual-cased-v2", SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')),
        ("stsb-xlm-r-multilingual", SentenceTransformer('sentence-transformers/stsb-xlm-r-multilingual')),
        ("neuralmind-bert-base-portuguese-cased", SentenceTransformer('neuralmind/bert-base-portuguese-cased'))
    ]
    return models

In [3]:
# Célula 3: Configuração do ChromaDB
def setup_chromadb(base_dir: str, models: List[Tuple[str, SentenceTransformer]]) -> Tuple[chromadb.PersistentClient, List[chromadb.Collection]]:
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    collections = []
    # Listar todas as coleções existentes uma única vez
    existing_collections = [col.name for col in chroma_client.list_collections()]

    for model_name, _ in models:
        collection_name = f"horarios-professores_{model_name}"

        # Deletar a coleção se existir
        if collection_name in existing_collections:
            print(f"Excluindo coleção existente: {collection_name}")
            chroma_client.delete_collection(name=collection_name)

        # Criar nova coleção
        print(f"Criando nova coleção: {collection_name}")
        collection = chroma_client.create_collection(
            name=collection_name,
            metadata={"description": f"Horários e informações usando modelo {model_name}"}
        )
        collections.append(collection)

    return chroma_client, collections

In [ ]:
# Célula 4: Função de Chunking Semântico
def create_text_chunks_semantico(json_data: Dict) -> List[Dict]:
    chunks = []
    professor = json_data['nome_professor']
    departamento = json_data['departamento']
    periodo = json_data['ano_periodo']

    # Chunk com informações básicas do professor
    chunks.append({
        'type': 'info_professor',
        'text': f"O professor {professor} pertence ao departamento {departamento} e está ativo no período {periodo}.",
        'metadata': {
            'professor': professor,
            'departamento': departamento,
            'periodo': periodo
        }
    })

    # Criar um dicionário para agrupar horários por disciplina
    disciplinas_horarios = {}

    # Coletar todos os horários por código de disciplina
    for dia, periodos in json_data['horarios'].items():
        for periodo_nome, horarios_dia in periodos.items():
            for horario_nome, info in horarios_dia.items():
                if info['codigo']:  # Se existe um código de disciplina
                    codigo = info['codigo']
                    if codigo not in disciplinas_horarios:
                        disciplinas_horarios[codigo] = {}

                    if dia not in disciplinas_horarios[codigo]:
                        disciplinas_horarios[codigo][dia] = []

                    disciplinas_horarios[codigo][dia].append({
                        'horario': horario_nome,
                        'inicio': info['inicio'],
                        'fim': info['fim'],
                        'sala': info['sala']
                    })

    # Para cada disciplina no JSON, criar um chunk organizado
    for codigo, disciplinas in json_data['disciplinas'].items():
        for disc in disciplinas:
            nome_disciplina = disc['nome']
            turma = disc['turma']
            enquadramento = disc['enquadramento']

            # Texto base da disciplina
            texto_disciplina = (
                f"O professor {professor} ministra a disciplina {nome_disciplina} ({codigo}) "
                f"para a turma {turma} no formato {enquadramento}."
            )

            # Adicionar horários organizados por dia
            if codigo in disciplinas_horarios:
                texto_horarios = "\nHorários:"
                for dia, horarios in sorted(disciplinas_horarios[codigo].items()):
                    # Ordenar horários pelo nome (m1, m2, etc.)
                    horarios_ordenados = sorted(horarios, key=lambda x: x['horario'])

                    # Formatar horários do dia
                    horarios_texto = [
                        f"{h['horario']} ({h['inicio']}-{h['fim']})"
                        for h in horarios_ordenados
                    ]

                    # Adicionar sala se disponível
                    sala = next((h['sala'] for h in horarios_ordenados if h['sala']), "")
                    sala_texto = f" - Sala {sala}" if sala else ""

                    texto_horarios += f"\n- {dia.capitalize()}: {', '.join(horarios_texto)}{sala_texto}"

            else:
                texto_horarios = ""

            # Criar o chunk completo
            chunks.append({
                'type': 'disciplina',
                'text': texto_disciplina + texto_horarios,
                'metadata': {
                    'professor': professor,
                    'departamento': departamento,
                    'periodo': periodo,
                    'codigo': codigo,
                    'nome_disciplina': nome_disciplina,
                    'turma': turma
                }
            })

    return chunks

In [ ]:
# Célula 5: Processamento dos Arquivos JSON
def process_json_files(base_dir: str, chroma_client: chromadb.PersistentClient,
                      models: List[Tuple[str, SentenceTransformer]],
                      collections: List[chromadb.Collection]):
    json_dir = os.path.normpath(os.path.join(base_dir, "..", "Webscraping", "Files", "JSON", "2025-1"))

    if not os.path.exists(json_dir):
        print(f"Erro: Diretório base '{json_dir}' não encontrado.")
        return {}

    chunks_por_professor = {}
    id_counter = 0

    # Coletar todos os chunks
    all_chunks = []
    for dept_name in os.listdir(json_dir):
        dept_dir = os.path.join(json_dir, dept_name)
        if os.path.isdir(dept_dir):
            print(f"Processando departamento: {dept_name}")

            for file_name in os.listdir(dept_dir):
                if file_name.lower().endswith(".json"):
                    json_file = os.path.join(dept_dir, file_name)
                    try:
                        print(f"Lendo arquivo JSON: {json_file}")
                        with open(json_file, 'r', encoding='utf-8') as f:
                            data = json.load(f)

                        professor = data['nome_professor']
                        print(f"Processando professor: {professor}")

                        chunks = create_text_chunks_semantico(data)
                        print(f"Criados {len(chunks)} chunks para {professor}")
                        all_chunks.extend(chunks)
                        chunks_por_professor[professor] = chunks

                    except Exception as e:
                        print(f"Erro ao processar {file_name}: {str(e)}")

    # Processar chunks em lotes
    batch_size = 100  # Ajuste conforme necessário

    # Criar embeddings com cada modelo e adicionar às respectivas coleções
    print("\nGerando embeddings e adicionando às coleções...")
    for (model_name, model), collection in zip(models, collections):
        print(f"\nProcessando modelo: {model_name}")

        # Processar em lotes
        for i in range(0, len(all_chunks), batch_size):
            batch_chunks = all_chunks[i:i + batch_size]

            try:
                # Preparar lotes de dados
                texts = [chunk['text'] for chunk in batch_chunks]
                metadatas = [chunk['metadata'] for chunk in batch_chunks]
                ids = [f"doc_{j}" for j in range(id_counter, id_counter + len(batch_chunks))]

                # *** FILTRAR TEXTOS VAZIOS ***
                valid_indices = [i for i, t in enumerate(texts) if t.strip()]  # Remove espaços em branco
                texts = [texts[i] for i in valid_indices]
                metadatas = [metadatas[i] for i in valid_indices]
                ids = [ids[i] for i in valid_indices]

                if not texts:
                    print("Todos os textos do lote estão vazios, pulando lote.")
                    continue

                # Gerar embeddings em lote
                try:
                    embeddings = model.encode(texts, convert_to_tensor=False)  # Importante: convert_to_tensor=False
                    embeddings = np.array(embeddings)  # Converter para NumPy array
                except Exception as e:
                    print(f"Erro ao gerar embeddings: {str(e)}")
                    continue

                # *** CHECAR SE HÁ NaN NOS EMBEDDINGS ***
                if np.isnan(embeddings).any():
                    print(f"Aviso: Encontrados valores NaN nos embeddings do lote {i//batch_size}")
                    continue

                # Adicionar à coleção
                collection.add(
                    documents=texts,
                    embeddings=embeddings.tolist(),  # Converter para lista
                    metadatas=metadatas,
                    ids=ids
                )

                id_counter += len(batch_chunks)
                print(f"Adicionado lote de {len(batch_chunks)} chunks à coleção {collection.name}")

            except Exception as e:
                print(f"Erro ao processar lote na coleção {collection.name}: {str(e)}")
                continue

        print(f"Concluído modelo {model_name}")

    print("\nProcessamento concluído!")
    return chunks_por_professor

In [ ]:
# Célula 6: Execução do Pipeline
if __name__ == "__main__":
    # Configurar modelos de embedding
    models = setup_embedding_models()

    # Configurar ChromaDB e coleções
    base_dir = os.getcwd()
    chroma_client, collections = setup_chromadb(base_dir, models)

    # Processar todos os arquivos JSON
    chunks_por_professor = process_json_files(base_dir, chroma_client, models, collections)

In [ ]:
# Célula: Listagem Simples de Coleções
import chromadb
from pathlib import Path

def simple_list_collections(base_dir: str) -> None:
    """
    Lista apenas os nomes das coleções existentes no ChromaDB.
    """
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    collections = chroma_client.list_collections()

    if not collections:
        print("Nenhuma coleção encontrada no ChromaDB.")
        return

    print("\nColeções encontradas:")
    for collection in collections:
        print(f"- {collection.name}")
    print(f"\nTotal: {len(collections)} coleções")

# Exemplo de uso
if __name__ == "__main__":
    base_dir = os.getcwd()  # ou especifique o diretório base
    simple_list_collections(base_dir)

In [ ]:
def check_collections_content(base_dir: str):
    """
    Verifica detalhadamente o conteúdo de cada coleção
    """
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    collections = chroma_client.list_collections()
    print(f"Total de coleções: {len(collections)}")

    for collection in collections:
        print(f"\nColeção: {collection.name}")
        try:
            # Tentar obter a contagem de documentos
            count = collection.count()
            print(f"Número de documentos: {count}")

            if count > 0:
                # Tentar obter alguns documentos de exemplo
                results = collection.get(limit=5)
                print("\nPrimeiros 5 documentos:")
                for i, (doc, meta) in enumerate(zip(results['documents'], results['metadatas'])):
                    print(f"\nDocumento {i+1}:")
                    print(f"Texto: {doc[:100]}...")  # Primeiros 100 caracteres
                    print(f"Metadata: {meta}")
        except Exception as e:
            print(f"Erro ao acessar coleção: {str(e)}")

# Execute esta função para verificar o conteúdo
check_collections_content(os.getcwd())

In [ ]:
import os
from pathlib import Path
import chromadb
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.decomposition import PCA
import traceback

def visualize_embeddings_pca(base_dir: str, interactive: bool = True) -> None:
    """
    Visualiza os embeddings de todas as coleções usando PCA.
    """
    try:
        chroma_dir = Path(base_dir) / "chroma_db"
        chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

        collections = chroma_client.list_collections()
        n_collections = len(collections)

        if n_collections == 0:
            print("Nenhuma coleção encontrada!")
            return

        # Configurar o layout dos subplots
        n_rows = (n_collections + 1) // 2
        n_cols = 2 if n_collections > 1 else 1

        fig = make_subplots(
            rows=n_rows,
            cols=n_cols,
            subplot_titles=[col.name.replace('horarios-professores_', '') for col in collections],
            vertical_spacing=0.15,
            horizontal_spacing=0.1
        )

        valid_collections = 0

        for idx, collection in enumerate(collections, 1):
            print(f"\nProcessando coleção: {collection.name}")

            try:
                # Obter contagem de documentos
                count = collection.count()
                print(f"Número total de documentos: {count}")

                if count == 0:
                    print(f"Aviso: Coleção {collection.name} está vazia")
                    continue

                # Obter todos os documentos
                results = collection.get(
                    include=['embeddings', 'documents', 'metadatas'],
                    limit=count
                )

                embeddings = results['embeddings']
                metadatas = results['metadatas']

                print(f"Número de embeddings recuperados: {len(embeddings)}")

                # Verificar o primeiro embedding para debug
                if len(embeddings) > 0:
                    print(f"Dimensões do primeiro embedding: {np.array(embeddings[0]).shape}")

                # Converter embeddings para numpy arrays e filtrar inválidos
                valid_embeddings = []
                valid_metadatas = []

                for i, (emb, meta) in enumerate(zip(embeddings, metadatas)):
                    try:
                        arr = np.array(emb, dtype=np.float64)
                        if arr.ndim == 1 and arr.size > 0 and not np.isnan(arr).any():
                            valid_embeddings.append(arr)
                            valid_metadatas.append(meta)
                    except Exception as e:
                        print(f"Erro ao processar embedding {i}: {str(e)}")
                        continue

                if not valid_embeddings:
                    print(f"Nenhum embedding válido encontrado na coleção {collection.name}")
                    continue

                print(f"Embeddings válidos encontrados: {len(valid_embeddings)}")

                # Converter para array 2D
                embeddings_array = np.vstack(valid_embeddings)
                print(f"Shape do array de embeddings: {embeddings_array.shape}")

                # Aplicar PCA
                pca = PCA(n_components=2)
                embeddings_2d = pca.fit_transform(embeddings_array)

                # Calcular variância explicada
                var_ratio = pca.explained_variance_ratio_
                var_explained = f"Variância explicada: {var_ratio[0]:.2%} (PC1), {var_ratio[1]:.2%} (PC2)"

                # Criar DataFrame
                df = pd.DataFrame({
                    'PC1': embeddings_2d[:, 0],
                    'PC2': embeddings_2d[:, 1],
                    'professor': [m['professor'] for m in valid_metadatas],
                    'tipo': [m.get('type', 'info_professor') for m in valid_metadatas],
                    'departamento': [m['departamento'] for m in valid_metadatas],
                    'disciplina': [m.get('nome_disciplina', '') for m in valid_metadatas]
                })

                # Calcular posição no subplot
                valid_collections += 1
                row = (valid_collections - 1) // 2 + 1
                col = 2 if valid_collections % 2 == 0 else 1

                # Criar scatter plot
                scatter = go.Scatter(
                    x=df['PC1'],
                    y=df['PC2'],
                    mode='markers',
                    marker=dict(
                        size=8,
                        color=pd.factorize(df['tipo'])[0],
                        colorscale='Viridis',
                        showscale=False
                    ),
                    text=[f"Professor: {p}<br>Departamento: {d}<br>Tipo: {t}<br>Disciplina: {disc}"
                          for p, d, t, disc in zip(df['professor'], df['departamento'], df['tipo'], df['disciplina'])],
                    hoverinfo='text',
                    name=collection.name
                )

                fig.add_trace(scatter, row=row, col=col)

                # Adicionar texto com variância explicada
                fig.add_annotation(
                    text=var_explained,
                    xref=f"x{valid_collections}", yref=f"y{valid_collections}",
                    x=0.5, y=-0.15,
                    showarrow=False,
                    font=dict(size=10),
                    xanchor='center'
                )

                # Atualizar layout dos eixos
                fig.update_xaxes(title_text="PC1", row=row, col=col)
                fig.update_yaxes(title_text="PC2", row=row, col=col)

            except Exception as e:
                print(f"Erro ao processar coleção {collection.name}:")
                print(traceback.format_exc())
                continue

        if valid_collections == 0:
            print("\nNenhuma coleção válida encontrada para visualização!")
            return

        # Atualizar layout geral
        fig.update_layout(
            title_text="Visualização de Embeddings (PCA)",
            height=400 * ((valid_collections + 1) // 2),
            width=1200,
            showlegend=False,
            template="plotly_white"
        )

        # Salvar e mostrar
        try:
            fig.write_html("embeddings_visualization_pca.html")
            print("\nVisualização salva em embeddings_visualization_pca.html")

            if not interactive:
                try:
                    import kaleido
                    fig.write_image("embeddings_visualization_pca.png", scale=2)
                    print("Visualização salva em embeddings_visualization_pca.png")
                except ImportError:
                    print("Para salvar como PNG, instale o pacote kaleido: pip install -U kaleido")
        except Exception as e:
            print(f"Erro ao salvar visualizações: {str(e)}")

        if interactive:
            fig.show()

    except Exception as e:
        print("Erro geral na função:")
        print(traceback.format_exc())

# Chamada direta da função
base_dir = os.getcwd()
visualize_embeddings_pca(base_dir, interactive=True)

In [ ]:
import os
from pathlib import Path
import chromadb
import numpy as np
import pandas as pd
import plotly.express as px
from sklearn.decomposition import PCA

def visualize_embeddings_pca_turma(base_dir: str, modelo: str = None, interactive: bool = True) -> None:
    """
    Visualiza os embeddings de uma coleção usando PCA, colorindo por turma.
    Se modelo for None, usa a primeira coleção encontrada.
    """
    chroma_dir = Path(base_dir) / "chroma_db"
    chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

    # Seleciona a coleção
    collections = chroma_client.list_collections()
    if not collections:
        print("Nenhuma coleção encontrada!")
        return

    if modelo:
        collection = next((c for c in collections if modelo in c.name), None)
        if not collection:
            print(f"Modelo '{modelo}' não encontrado nas coleções.")
            return
    else:
        collection = collections[0]
        print(f"Usando coleção: {collection.name}")

    # Obter todos os documentos
    count = collection.count()
    results = collection.get(include=['embeddings', 'metadatas'], limit=count)
    embeddings = results['embeddings']
    metadatas = results['metadatas']

    # Filtrar embeddings válidos
    valid_embeddings = []
    valid_metadatas = []
    for emb, meta in zip(embeddings, metadatas):
        arr = np.array(emb)
        if arr.ndim == 1 and arr.size > 0 and not np.isnan(arr).any():
            valid_embeddings.append(arr)
            valid_metadatas.append(meta)

    if not valid_embeddings:
        print("Nenhum embedding válido encontrado.")
        return

    embeddings_array = np.vstack(valid_embeddings)
    pca = PCA(n_components=2)
    embeddings_2d = pca.fit_transform(embeddings_array)

    # Criar DataFrame
    df = pd.DataFrame({
        'PC1': embeddings_2d[:, 0],
        'PC2': embeddings_2d[:, 1],
        'professor': [m['professor'] for m in valid_metadatas],
        'departamento': [m['departamento'] for m in valid_metadatas],
        'tipo': [m.get('type', 'info_professor') for m in valid_metadatas],
        'disciplina': [m.get('nome_disciplina', '') for m in valid_metadatas],
        'turma': [m.get('turma', 'desconhecida') for m in valid_metadatas]  # <-- aqui!
    })

    # Visualização com cor por turma
    fig = px.scatter(
        df, x='PC1', y='PC2',
        color='departamento',
        hover_data=['professor', 'departamento', 'tipo', 'disciplina', 'turma'],
        title=f'Embeddings PCA colorido por Turma ({collection.name})'
    )

    if interactive:
        fig.show()
    else:
        fig.write_html("embeddings_pca_turma.html")
        print("Visualização salva em embeddings_pca_turma.html")

# Exemplo de uso:
base_dir = os.getcwd()
# Se quiser um modelo específico, passe o nome (ou parte do nome) em modelo=...
# Exemplo: modelo="distiluse-base-multilingual-cased-v2"
visualize_embeddings_pca_turma(base_dir, modelo=None, interactive=True)

In [ ]:
# Célula: Listar todos os metadados com o campo 'nome_disciplina' da coleção horarios-professores_all-MiniLM-L6-v2

import chromadb
from pathlib import Path
import os

# Defina o diretório base e inicialize o cliente ChromaDB
base_dir = os.getcwd()
chroma_dir = Path(base_dir) / "chroma_db"
chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

# Nome da coleção desejada
collection_name = "horarios-professores_all-MiniLM-L6-v2"

try:
    collection = chroma_client.get_collection(collection_name)
except Exception as e:
    print(f"Erro ao carregar a coleção: {e}")
    collection = None

if collection:
    count = collection.count()
    print(f"Total de documentos na coleção: {count}")

    # Buscar todos os metadados
    results = collection.get(include=['metadatas'], limit=count)
    metadatas = results['metadatas']

    disciplinas = []
    for meta in metadatas:
        if 'nome_disciplina' in meta:
            disciplinas.append(meta['nome_disciplina'])

    print(f"\nTotal de metadados com 'nome_disciplina': {len(disciplinas)}")
    for i, nome in enumerate(disciplinas, 1):
        print(f"{i}: {nome}")
else:
    print("Coleção não encontrada.")


In [ ]:
# Célula: Listar todos os professores únicos da coleção horarios-professores_all-MiniLM-L6-v2

import chromadb
from pathlib import Path
import os

# Defina o diretório base e inicialize o cliente ChromaDB
base_dir = os.getcwd()
chroma_dir = Path(base_dir) / "chroma_db"
chroma_client = chromadb.PersistentClient(path=str(chroma_dir))

# Nome da coleção desejada
collection_name = "horarios-professores_all-MiniLM-L6-v2"

try:
    collection = chroma_client.get_collection(collection_name)
except Exception as e:
    print(f"Erro ao carregar a coleção: {e}")
    collection = None

if collection:
    count = collection.count()
    print(f"Total de documentos na coleção: {count}")

    # Buscar todos os metadados
    results = collection.get(include=['metadatas'], limit=count)
    metadatas = results['metadatas']

    professores = set()
    for meta in metadatas:
        if 'professor' in meta:
            professores.add(meta['professor'])

    print(f"\nTotal de professores únicos: {len(professores)}")
    for i, nome in enumerate(sorted(professores), 1):
        print(f"{i}: {nome}")
else:
    print("Coleção não encontrada.")

In [4]:
# Célula: Exibir todos os chunks das coleções para o professor

professor_nome = "Evando Carlos Pizzini"
colecoes = [
    ("horarios-professores_all-MiniLM-L6-v2", 384)
]

for collection_name, emb_size in colecoes:
    print(f"\n{'='*80}")
    print(f"Coleta: {collection_name}")
    print(f"{'='*80}")
    try:
        collection = chroma_client.get_collection(collection_name)
    except Exception as e:
        print(f"Erro ao carregar a coleção: {e}")
        continue

    if collection:
        results = collection.query(
            query_embeddings=[[0.0]*emb_size],  # Dummy embedding
            where={"professor": professor_nome},
            n_results=1000,
            include=["documents", "metadatas"]
        )

        docs = results.get("documents", [[]])[0]
        metas = results.get("metadatas", [[]])[0]

        print(f"Total de chunks encontrados para o professor '{professor_nome}': {len(docs)}\n")
        for i, (doc, meta) in enumerate(zip(docs, metas), 1):
            print(f"Chunk {i}:")
            print("Texto:", doc.strip())
            print("Metadados:", meta)
            print("-" * 60)
    else:
        print("Coleção não encontrada.")


Coleta: horarios-professores_all-MiniLM-L6-v2
Total de chunks encontrados para o professor 'Evando Carlos Pizzini': 5

Chunk 1:
Texto: O professor Evando Carlos Pizzini ministra a disciplina Fundamentos De Estruturas De Dados (COM1009) para a turma CC2 no formato Presencial.
Horários:
- Quinta-feira: m4 (10:20-11:10), m5 (11:10-12:00) - Sala I25
- Terca-feira: m4 (10:20-11:10), m5 (11:10-12:00) - Sala I25
Metadados: {'turma': 'CC2', 'professor': 'Evando Carlos Pizzini', 'nome_disciplina': 'Fundamentos De Estruturas De Dados', 'periodo': '2025/1', 'departamento': 'DACOM', 'codigo': 'COM1009'}
------------------------------------------------------------
Chunk 2:
Texto: O professor Evando Carlos Pizzini pertence ao departamento DACOM e está ativo no período 2025/1.
Metadados: {'professor': 'Evando Carlos Pizzini', 'departamento': 'DACOM', 'periodo': '2025/1'}
------------------------------------------------------------
Chunk 3:
Texto: O professor Evando Carlos Pizzini ministra a discipli